<a href="https://colab.research.google.com/github/bnsreenu/python_for_microscopists/blob/master/314_How_to_code_the_genetic_algorithm_in_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://youtu.be/EJeTWRP3Bd0

# What is Genetic algorithm and How to code it in python?<br>
The genetic algorithm is a stochastic method for function optimization inspired by the process of natural evolution - select parents to create children using the crossover and mutation processes.
<p>
The code is an implementation of the genetic algorithm for optimization. The algorithm is used to find the optimum features for the maximum possibility to be still married after 15 years. The algorithm consists of the following steps:<p>

1. Initialize a population of binary bitstrings with random values.
2. Decode the binary bitstrings into numerical values, and evaluate the fitness (the objective function) for each individual in the population.
3. Select the best individuals from the population using tournament selection based on the fitness scores.
4. Create new offsprings from the selected individuals using the crossover operation.
5. Apply the mutation operation on the offsprings to maintain diversity in the population.
6. Repeat steps 2 to 5 until a stopping criterion is met.

<p>

The implementation includes functions for decoding, selection, crossover, and mutation.

In [198]:
from numpy.random import randint
from numpy.random import rand
import math

In [199]:
# The objective function is the formula for calculating the probability of still being married after 15 years according to some features.
def objective(x):
  y = 50*((x[0]/x[1])*((x[2]+x[3])/(x[4]+5))*x[5]*(x[5]/(x[5]+2))**400)**(1/15)
  return y

In [200]:
# The decode function decodes binary bitstrings to numbers for each input and scales the value to fall within defined bounds
def decode(bounds, n_bits, bitstring):
	"""
	This function decodes binary bitstrings to numbers for each input and scales the value to fall within defined bounds.

	Parameters:
		bounds (list): A list of tuples representing the lower and upper bounds for each value to be decoded.
		n_bits (int): The number of bits used to represent each value.
		bitstring (list): A binary bitstring to be decoded.

	Returns:
		decoded (list): A list of decoded values.
	"""
	decoded = []  # Create an empty list to hold the decoded values
	largest = 2**n_bits  # The largest value that can be represented with the given number of bits
	for i in range(len(bounds)):
		# Extract the substring for the current value
		start, end = i * n_bits, (i * n_bits) + n_bits  # Define the start and end indices of the substring
		substring = bitstring[start:end]  # Extract the substring
		# Convert the substring to a string of characters
		chars = ''.join([str(s) for s in substring])  # Join the values in the substring together into a string of characters
		# Convert the string of characters to an integer
		integer = int(chars, 2)  # Convert the binary number string into an integer
		# Scale the integer to the desired range
		value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])  # Scale the integer to a value within the defined bounds
		# Store the decoded value
		decoded.append(value)
	return decoded

In [201]:
def selection(pop, scores, k=3):
    """
    Select the best individuals for the next generation based on their fitness (scores).
    This function randomly selects k individuals from the population and performs a tournament
    among them to choose the one with the best score.

    Parameters:
    pop (list): The population of individuals.
    scores (list): The fitness scores for each individual in the population.
    k (int, optional): The number of individuals to select from the population for the tournament.
                        Defaults to 3.

    Returns:
    individual: The best individual from the tournament.
    """
    # Randomly select one index from the population as the initial selection
    selection_ix = randint(len(pop))
    # Perform a tournament among k randomly selected individuals
    for ix in randint(0, len(pop), k-1):
        # Check if the current individual has a better score than the selected one
        if scores[ix] > scores[selection_ix]:
            # Update the selected individual if a better one is found
            selection_ix = ix
    # Return the best individual from the tournament
    return pop[selection_ix]

In [202]:
def crossover(p1, p2, r_cross):
    """
    Create two children from two parents using the crossover operation.
    The children are created by copying the parents, and recombination occurs
    if a random value is less than the crossover rate.

    Parameters:
    p1 (list): The first parent.
    p2 (list): The second parent.
    r_cross (float): The crossover rate.

    Returns:
    list: A list containing the two children.
    """
    # Children are copies of the parents by default
    c1, c2 = p1.copy(), p2.copy()
    # Check if recombination should occur
    if rand() < r_cross:
        # Select a crossover point (not at the end of the string)
        pt = randint(1, len(p1)-2)
        # Perform crossover in the children
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    # Return the two children
    return [c1, c2]

In [203]:
#The crossover process can generate offsprings that are very similar to the parents. This might cause a new generation with low diversity.
# The mutation process solves this problem by changing the value of some features in the offspring at random.


import random

def mutation(bitstring, r_mut):
    """
    The mutation process changes the value of some features in the offspring at random to maintain the diversity in the population.
    A standard value for the mutation rate is 1/m where m is the number of features.

    Parameters:
    bitstring (list): A list of binary values representing the offspring
    r_mut (float): The mutation rate, typically 1/m where m is the number of features

    Returns:
    list: The modified bitstring after mutation

    """
    rand = random.random
    for i in range(len(bitstring)):
        # Check for a mutation
        if rand() < r_mut:
            # Flip the bit
            bitstring[i] = 1 - bitstring[i]
    return bitstring

In [204]:
### Putting all together into our Genetic algorithm that runs until it finds the best
#The whole fitness assignment, selection, recombination, and mutation process is
#repeated until a stopping criterion is satisfied.
#Each generation is likely to be more adapted to the environment than the old one.

# genetic algorithm implementation
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
    """
    The genetic algorithm that finds the optimal solution by performing the fitness assignment, selection, recombination, and mutation process repeatedly.
    Each iteration, the solution is more adapted to the environment.

    Parameters
    ----------
    objective: function
        The objective function that needs to be optimized.
    bounds: list of tuples
        The bounds of the solution.
    n_bits: int
        The number of bits used to encode the solution.
    n_iter: int
        The number of iterations to perform.
    n_pop: int
        The size of the population.
    r_cross: float
        The crossover rate.
    r_mut: float
        The mutation rate.

    Returns
    -------
    list
        The best solution and its evaluation.
    """
    # initialize the population with observed bitstrings
    pop_int = [[618,26,33,30,0,50],[218,47,67,45,0,84],[258,44,29,29,0,120],[589,25,34,30,0,24],[16,1,54,35,3,36],[36,1,26,22,5,20],[529,48,28,28,1,28],[513,23,29,28,1,14],
           [1299,145,38,38,0,17],[1083,378,48,39,2,60],[170,74,38,26,1,72],[3,3,3,30,0,120],[192,31,29,22,2,36],[168,29,28,25,0,24],[187,45,29,26,1,36],[722,131,29,42,1,21],
           [539,67,39,35,0,12],[986,105,27,37,0,10],[576,53,35,36,0,8],[540,55,31,23,1,12],[57,17,28,36,1,26],[702,262,36,31,0,24],[3,1,34,28,1,22],[156,29,37,28,1,12],
           [229,57,32,32,0,13],[205,93,24,31,0,24],[205,93,29,26,0,24],[1299,145,27,22,1,8],
           [104,39,36,23,3,24],[339,158,46,27,0,14],[141,54,28,23,0,12],[2,15,36,25,1,24],[13,21,37,30,3,36],[215,72,26,30,2,8],[35,30,24,21,3,24],[101,86,36,31,4,18],
           [24,53,42,30,0,14],[364,207,30,29,2,6],[3,5,28,36,0,10],[24,23,29,24,0,6]]
    pop, subpop=[], []
    dgt=lambda x : ''.join(reversed( [str((x >> j) & 1) for j in range(16)] ) )
    for i in range(len(pop_int)):
        for j in range(6):
            x=[int(k) for k in dgt(pop_int[i][j])]
            subpop+=x
        pop.append(subpop)
        subpop=[]

    # track the best solution found so far
    best, best_eval = 0, objective(decode(bounds, n_bits, pop[39]))

    # iterate over generations
    for gen in range(n_iter):
        # decode the population
        decoded = [decode(bounds, n_bits, p) for p in pop]
        # evaluate all candidates in the population
        scores = [objective(d) for d in decoded]
        # check for a new best solution
        for i in range(n_pop):
            if scores[i] > best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %f" % (gen,  decoded[i], scores[i]))

        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]

        # create the next generation - children
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i + 1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace the population
        pop = children
    return [best, best_eval]




In [205]:

# define range for input
bounds = [[0.1, 2000.0], [0.1, 500.0], [20.0, 70.0], [20.0, 50.0], [0.1, 10.0], [1.0, 132.0]]
# define the total iterations
n_iter = 1000
# bits per variable
n_bits = 16
# define the population size
n_pop = 40
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))
# perform the genetic algorithm search
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('###########################################################')
decoded = decode(bounds, n_bits, best)
print('The result is (%s) with a score of %f' % (decoded, score))

>0, new best f([18.958920288085938, 0.2983245849609375, 20.025177001953125, 20.01373291015625, 0.1, 1.099945068359375]) = 0.000000
>0, new best f([6.752499389648437, 0.45850982666015627, 20.051116943359375, 20.020599365234375, 0.1, 1.16790771484375]) = 0.000000
>0, new best f([7.973141479492187, 0.43562622070312496, 20.022125244140625, 20.013275146484375, 0.1, 1.2398681640625]) = 0.000000
>1, new best f([1.9919952392578126, 2.0985015869140624, 20.0274658203125, 20.014190673828125, 0.1, 1.5596923828125]) = 0.000000
>1, new best f([5.95908203125, 0.33646392822265625, 20.022125244140625, 20.01373291015625, 0.10045318603515625, 33.82196044921875]) = 18.986288
>1, new best f([16.578668212890626, 258.28047027587894, 20.018310546875, 20.014190673828125, 0.2571044921875, 66.5479736328125]) = 28.632526
>2, new best f([5.95908203125, 0.33646392822265625, 20.022125244140625, 20.01373291015625, 0.10045318603515625, 50.70867919921875]) = 32.175006
>3, new best f([18.958920288085938, 8.4753997802734

In [175]:
pop_int = [[618,26,33,30,0,50],[218,47,67,45,0,84],[258,44,29,29,0,120],[589,25,34,30,0,24],[16,1,54,35,3,36],[36,1,26,22,5,20],[529,48,28,28,1,28],[513,23,29,28,1,14],
           [1299,145,38,38,0,17],[1083,378,48,39,2,60],[170,74,38,26,1,72],[3,3,3,30,0,120],[192,31,29,22,2,36],[168,29,28,25,0,24],[187,45,29,26,1,36],[722,131,29,42,1,21],
           [539,67,39,35,0,12],[986,105,27,37,0,10],[576,53,35,36,0,8],[540,55,31,23,1,12],[57,17,28,36,1,26],[702,262,36,31,0,24],[3,1,34,28,1,22],[156,29,37,28,1,12],
           [229,57,32,32,0,13],[205,93,24,31,0,24],[205,93,29,26,0,24],[1299,145,27,22,1,8],
           [104,39,36,23,3,24],[339,158,46,27,0,14],[141,54,28,23,0,12],[2,15,36,25,1,24],[13,21,37,30,3,36],[215,72,26,30,2,8],[35,30,24,21,3,24],[101,86,36,31,4,18],
           [24,53,42,30,0,14],[364,207,30,29,2,6],[3,5,28,36,0,10],[24,23,29,24,0,6]]
pop, subpop=[], []
dgt=lambda x : ''.join(reversed( [str((x >> j) & 1) for j in range(16)] ) )
for i in range(len(pop_int)):
    for j in range(6):
        x=[int(k) for k in dgt(pop_int[i][j])]
        subpop+=x
    pop.append(subpop)
    subpop=[]
print(pop[4])


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
